In [ ]:

# 📦 Importing Required Libraries
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


In [ ]:

# 📁 Dataset Preparation
import zipfile
import urllib.request

dataset_url = "https://storage.googleapis.com/wandb_datasets/nature_12K.zip"
dataset_zip = "nature_12K.zip"
extract_path = "./nature_dataset"

# Download and extract dataset if not already done
if not os.path.exists(extract_path):
    urllib.request.urlretrieve(dataset_url, dataset_zip)
    with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    os.remove(dataset_zip)
    print("Dataset ready at:", extract_path)
else:
    print("Dataset already available.")


In [ ]:

# 🧪 Custom Data Transformations
def image_transforms():
    return transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])


In [ ]:

# 🔄 Load and Split Data
def get_dataloaders(data_dir, batch_size=32, val_ratio=0.2):
    dataset = datasets.ImageFolder(data_dir, transform=image_transforms())
    total = len(dataset)
    val_size = int(total * val_ratio)
    train_size = total - val_size

    train_ds, val_ds = random_split(dataset, [train_size, val_size])
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, dataset.classes


In [ ]:

# 🧠 Create ResNet Model
def build_model(num_classes):
    model = models.resnet18(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model.to(device)


In [ ]:

# 🏋️ Training Procedure
def train_model(model, dataloader, optimizer, criterion, epochs=5):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        correct = 0
        total = 0

        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        acc = correct / total * 100
        print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss:.4f}, Accuracy: {acc:.2f}%")


In [ ]:

# 📊 Evaluation Function
def evaluate_model(model, dataloader, class_names):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for imgs, lbls in dataloader:
            imgs, lbls = imgs.to(device), lbls.to(device)
            outputs = model(imgs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(lbls.cpu().numpy())

    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, xticklabels=class_names, yticklabels=class_names, fmt='d', cmap="Blues")
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.show()


In [ ]:

# 🚀 Run Training Pipeline
DATA_DIR = "./nature_dataset/inaturalist_12K/train"
BATCH_SIZE = 32
EPOCHS = 5

train_loader, val_loader, labels = get_dataloaders(DATA_DIR, BATCH_SIZE)
cnn = build_model(len(labels))

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn.fc.parameters(), lr=0.001)

train_model(cnn, train_loader, optimizer, loss_fn, EPOCHS)
evaluate_model(cnn, val_loader, labels)
